In [75]:
import pandas as pd
pd.set_option("display.max_row", None)

In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
deal_data = {
    "name":"AggRule by condition"
    ,"dates":{"cutoff":"2021-06-01"
              ,"closing":"2021-07-15"
              ,"firstPay":"2021-08-26"
              ,"payFreq":["DayOfMonth",20]
              ,"poolFreq":"MonthEnd"
              ,"stated":"2030-01-01"}
    ,"pool":{'assets':[["Mortgage"
                        ,{"originBalance":2400,"originRate":["fix",0.045],"originTerm":30
                          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                          ,{"currentBalance":2400
                          ,"currentRate":0.08
                          ,"remainTerm":30
                          ,"status":"current"}]]}
    ,"accounts":{"prinAcc":{"balance":0},"intAcc":{"balance":0}}
    ,"bonds":{"A1":{"balance":800
                 ,"rate":0.07
                 ,"originBalance":800
                 ,"originRate":0.07
                 ,"startDate":"2020-01-03"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2020-01-03"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{"trusteeFee":{"type":{"fixFee":30}}}
    ,"collect":[["CollectedInterest","intAcc"]
               ,["CollectedPrincipal","prinAcc"]
               ,["CollectedRecoveries","prinAcc"]
               ,["CollectedPrepayment","prinAcc"]]
    ,"waterfall":{"default":[
         ["If",["status", "Defaulted"]
              ,["transfer","intAcc","prinAcc"] ]
         ,["payFee","intAcc",['trusteeFee']]         
         ,["accrueAndPayInt","intAcc",["A1"]]
         ,["payPrin","prinAcc",["A1"]]
         ,["payPrin","prinAcc",["B"]]
         ,["payPrinResidual","prinAcc",["B"]]
     ]}
    ,"status":("PreClosing","Amortizing")
}

In [92]:
from absbox import API,mkDeal
localAPI = API("http://localhost:8081",check=False)

trigger = {
    "AfterCollect":
      {"poolDef":
        {"condition":[("cumPoolDefaultedBalance",),">",20]
        ,"effects":("newStatus","Defaulted")
        ,"status":False
        ,"curable":False}
      }
}

deal = mkDeal(deal_data | {"triggers":trigger} )

r = localAPI.run(deal
                 ,poolAssump = ("Pool"
                                ,("Mortgage"
                                 ,{"CDR":0.02} ,{"CPR":0.02}, None, None)
                                 ,None
                                 ,None)
                 ,runAssump = []
                 ,read=True)

#r['pool']['flow']

⠋ Connecting engine server -> http://localhost:8081
✅Connected, local lib:0.20.2, server:0.22.7
⠋0m


In [93]:
r['accounts']['intAcc']

,balance,change,memo
date,,,
2021-07-15,28.15,28.15,<PoolInflow:CollectedInterest>
2021-07-15,28.15,0.00,<PoolInflow:CollectedInterest>
2021-08-26,0.00,-28.15,<SeqPayFee:trusteeFee>
2021-08-26,0.00,0.00,<PayInt:A1>
2021-08-31,13.28,13.28,<PoolInflow:CollectedInterest>
2021-09-20,11.43,-1.85,<SeqPayFee:trusteeFee>
2021-09-20,1.86,-9.57,<PayInt:A1>
2021-09-30,14.60,12.74,<PoolInflow:CollectedInterest>
2021-10-20,14.60,0.00,<SeqPayFee:trusteeFee>


In [86]:
r['result']['status']

,Date,From,To
0,2021-07-15,PreClosing,Amortizing
1,2021-12-31,Amortizing,Defaulted
